In [4]:
!pip install -q langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [5]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.7 MB/s eta 0:00:00


In [7]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS


embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")


In [1]:
!pip install -q youtube_transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 9.8 MB/s eta 0:00:00


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

# step1 a -> Indexing document (Document Ingestion)

In [15]:
video_id = "E9lAeMz1DaM"


try:
  transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

  # flatten to the plain text
  transcript = " ".join(chunk["text"] for chunk in transcript_list)
  print(transcript)

except TranscriptsDisabled:
  print("No captions are availabe for this video")
except Exception as e:
  print(f"An error occurred: {e}")

i don't know what that dream is that you have i don't care how disappointing it might have been as you've been working toward that dream that that dream that you're holding in your mind that it's possible [Music] you


In [16]:
transcript_list

[{'text': "i don't know what that dream is that you",
  'start': 0.96,
  'duration': 3.04},
 {'text': 'have', 'start': 2.48, 'duration': 3.68},
 {'text': "i don't care how disappointing it might",
  'start': 4.0,
  'duration': 3.52},
 {'text': 'have been', 'start': 6.16, 'duration': 4.96},
 {'text': "as you've been working toward that dream",
  'start': 7.52,
  'duration': 5.039},
 {'text': "that that dream that you're holding in",
  'start': 11.12,
  'duration': 2.88},
 {'text': 'your mind', 'start': 12.559, 'duration': 5.0},
 {'text': "that it's possible", 'start': 14.0, 'duration': 3.559},
 {'text': '[Music]', 'start': 23.13, 'duration': 12.169},
 {'text': 'you', 'start': 37.68, 'duration': 2.08}]

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5)
chunks = splitter.create_documents([transcript])

In [14]:
print(len(chunks))

30


# 1b text splitting

In [17]:
splitter = splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5)
chunks = splitter.create_documents([transcript])

In [18]:
len(chunks)

5

In [19]:
chunks[3]

Document(metadata={}, page_content="that that dream that you're holding in your mind")

# Embedding Generation and storing in vector store

In [20]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
vector_store = FAISS.from_documents(chunks, embedding)


In [22]:
vector_store.index_to_docstore_id
# its a 10 sec video clip so

{0: 'ab21fc73-024c-42ac-9d83-868ca0be0bac',
 1: '192d9143-5d9d-4b52-85f2-9436d39fc612',
 2: '47bbff09-f694-4a45-90ca-3b6152b264d5',
 3: 'ce200abb-7415-4820-b6b3-d3f616d5fcb1',
 4: '9f2e631e-73af-49de-8780-6efb50ed5c66'}

In [23]:
vector_store.get_by_ids(["47bbff09-f694-4a45-90ca-3b6152b264d5"])

[Document(id='47bbff09-f694-4a45-90ca-3b6152b264d5', metadata={}, page_content="been as you've been working toward that dream")]

# steps 2 - Retriver

In [24]:
retriver = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [25]:
retriver

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e1f97bbb0d0>, search_kwargs={'k': 2})

In [26]:
retriver.invoke("what is dream")

[Document(id='ab21fc73-024c-42ac-9d83-868ca0be0bac', metadata={}, page_content="i don't know what that dream is that you have i"),
 Document(id='ce200abb-7415-4820-b6b3-d3f616d5fcb1', metadata={}, page_content="that that dream that you're holding in your mind")]

# step 3 :Augmentation

In [27]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [28]:
prompt = PromptTemplate(
    template="""
    you are helpful assistant.
    Answer only from the provided transcript contex.
    If the contex is insufficient , just say you dont know.

    {context}
    question: {question}

    """,
    input_variables=["context", "question"]
)

In [ ]:
question = "is the topic of ai is discussed in this video? if yes then what was discussed"
retrived_docs = prompt.invoke(question)

In [ ]:
retrived_docs

In [ ]:
context_text =  "\n\n".join(doc.page_content for doc in retrived_docs )

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question":question})

#step-4: Generation

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

# Building the Chains

In [29]:
from langchain_core.runnables import RunnablePassthrough,RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrived_docs):
  context_text =  "\n\n".join(doc.page_content for doc in retrived_docs )
  return context_text

In [ ]:
parallel_chain = RunnableParallel(
    {

    "context":retriver | RunnableLambda(format_docs),
    "question":RunnablePassthrough()
    }
)

In [ ]:
parallel_chain.invoke("what is dream")

In [30]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | parser

In [ ]:
main_chain.invoke("summarize the video in one line")